# <span style="color:darkred">Additive Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

#### Set path and parameters

In [14]:
import numpy as np
import math
%run DD_Utilities_sjt_al.ipynb
%run AS_Utilities.ipynb
iprint= 4

In [15]:
iparam= 4

In [16]:
if iparam == 0:
    n= 10
    p= 4 
    b= [1,2,4,7]
    e= [3,4,8,10] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [17]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [18]:
# This example shows the optional parameters for set_parameters
if iparam == 2:
    n= 12
    p= -1 #p should be a positive integer or -1
    b= [-1] # an evenly dispersed set of domains will be chosen
    e= [-1] #List of ends or integer representing overlap or [-1] to automaically choose overlap
    K= 6
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [19]:
#subdomains contained in others
if iparam == 3:
    n= 10
    p= 5 #p should be a positive integer or -1
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [20]:
#overlap does not connect all subdomains
if iparam == 4:
    n= 9
    p= 4 
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    alpha= 0.5
    if K > 500:
        raise ValueError("K is too large")

In [21]:
p,b,e= set_parameters(n,p,b,e)  #set_parameters creates a sample set of domains if the user does not fully specify it.
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 3, 6, 8] [2, 5, 7, 9]


#### Establish and solve forward system 

In [22]:
sread= False    #Read system of equations from csv files
swrite= True  #Write system to csv file

M,x,y,xstore= Linear_system(n,sread,swrite,iparam)
svpert=Make_svpert_add(n,p,sread,iparam)
if iprint >= 4:
    print(M)
    print('y')
    print(y)        
    print('x')
    print(x)
if swrite:
    save_to_csv(n,p,K,nb,nn,ne,M,y,svpert,iparam)

[[9.62215655e+00 8.32901422e-01 1.43700570e-01 2.00881488e-01
  8.58934016e-01 6.66734650e-01 7.91466856e-02 3.17815819e-01
  4.04658996e-01]
 [3.04248768e-01 9.98588149e+00 1.81891619e-01 7.19007097e-01
  6.61667818e-02 1.54954091e-01 7.92437200e-01 4.85667763e-03
  6.29124987e-01]
 [7.32549846e-01 1.21512673e-01 9.62133427e+00 4.21599101e-01
  5.05099066e-02 4.71345005e-01 2.37365521e-01 8.81531530e-01
  8.71632211e-02]
 [8.37433784e-01 7.79150010e-01 6.99772570e-01 9.99655558e+00
  8.82255646e-01 6.92654503e-01 6.08992558e-01 2.00266278e-01
  9.37670041e-01]
 [3.86422709e-02 6.21906064e-01 3.26074979e-01 7.47165717e-01
  9.66372305e+00 9.86810485e-01 4.83876451e-01 2.43806914e-01
  9.42411059e-01]
 [7.41041186e-01 6.23529911e-02 1.34402552e-01 2.35042984e-01
  9.14452012e-01 9.72070277e+00 2.68590906e-01 5.46808478e-01
  8.88525134e-01]
 [6.51928944e-01 5.23795680e-01 2.98566812e-01 3.62890447e-01
  6.83464700e-01 9.99462815e-01 9.68294718e+00 4.94016569e-01
  7.58993183e-01]
 [1.50

In [23]:
# ### Iterative strategy
# $u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

# <ins>Multiplicative<ins>
# $A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i}, \\
# B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n}, \\
# C_i= B_i A   \in \mathbb{R}^{n \times n},  \\
# D_i= (I-C_i) \in \mathbb{R}^{n \times n},  \\
# f_i= B_i f   \in \mathbb{R}^{n}, \; i=1,\dots,p.
# $

In [24]:
#Construct domains, R,A,B,C and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
# D= Dmatrices_multiplicative(C,n,p,iprint)
f= fvector(B,y,n,p,iprint)

Print f vector 
i= 0
[0.01919833 0.0332165  0.         0.         0.         0.
 0.         0.         0.        ]
i= 1
[0.         0.         0.0203346  0.01645151 0.07113271 0.
 0.         0.         0.        ]
i= 2
[0.         0.         0.         0.         0.         0.0592938
 0.05528696 0.         0.        ]
i= 3
[0.         0.         0.         0.         0.         0.
 0.         0.07901634 0.08139461]


#### Perform $K$  iterations computing subdomain contributions independently
$
u^{\{k+i/p\}} = u^{\{k+(i-1)/p\}} + B_i(f-Au^{\{k+(i-1)/p\}}) =D_i u^{\{k+(i-1)/p\}} + f_i \quad i=1,\dots,p$

In [25]:
#Perform D iteration
vstore= Citeration_additive(C,f,alpha,n,p,K,iprint)
if iprint >= 2:
    print('vstore')
    print(vstore)


TypeError: 'numpy.ndarray' object is not callable

#### Perform $K$ iterations using iteration matrices $S$ and $T$,  and  which compute the solution in all $p$ domains simultanously
Let $p=5$ and construct S and T, g and v such that

$S = \pmatrix{
 I   &  0   &   0   &  0   &   0   \cr
-D_2 &  I   &   0   &  0   &   0   \cr
 0   & -D_3 &   I   &  0   &   0   \cr
 0   &   0  &  -D_4 &  I   &   0   \cr
 0   &   0  &   0   & -D_5 &   I
} \in \mathbb{R}^{np \times np}$,
$T =
\pmatrix{
 0 & 0 & 0 & 0 & -D_1  \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0
} \in \mathbb{R}^{np \times np}$, 
$g =
\pmatrix{
f_1  \cr
f_2  \cr
f_3  \cr
f_4  \cr
f_5
}
\in \mathbb{R}^{np}$,
$v^{\{k\}} =
\pmatrix{
u^{\{(k-1)+1/5\}}  \cr
u^{\{(k-1)+2/5\}}  \cr
u^{\{(k-1)+3/5\}}  \cr
u^{\{(k-1)+4/5\}}  \cr
u^{\{k\}}
}
\in \mathbb{R}^{np}$,

In [ ]:
# Perform D iteration
# -------------------

# print('Construct D and g and solve \n')
D= Dmatrix_additive(C,alpha,n,p,iprint)
g= gvector_additive(f,alpha,n,p,iprint)

# print('Perform D iteration \n')
[wstore]= Diteration_additive(D,g,n,p,K,iprint)
if iprint >= 2:
    print('wstore')
    disp(wstore)

In [ ]:
# #### Perform $K$ iterations as single matrix solve

# $K=6$ iterations of multiplicative Schwarz can be written as the $npK$-dimensional system of equations $Uz=h$, where

# $U=
# \pmatrix{
#  S & 0 & 0 & 0 & 0 & 0   \cr
#  T & S & 0 & 0 & 0 & 0   \cr
#  0 & T & S & 0 & 0 & 0   \cr
#  0 & 0 & T & S & 0 & 0   \cr
#  0 & 0 & 0 & T & S & 0   \cr
#  0 & 0 & 0 & 0 & T & S
# }
# $
# ,
# $z=
# \pmatrix{
# v^{\{1\}}  \cr
# v^{\{2\}}  \cr
# v^{\{3\}}  \cr
# v^{\{4\}}  \cr
# v^{\{5\}}  \cr
# v^{\{6\}}
# }
# $
# ,
# $
# h=
# \pmatrix{
# g  \cr
# g  \cr
# g  \cr
# g  \cr
# g  \cr
# g
# }
# $
# .

In [ ]:
# Construct U and h
U= Umatrix_additive(D,n,K,iprint)
h= hvector_additive(g,n,K,iprint)

# Solve using Uz= h
zstore= Uhsolve_additive(U,h,n,p,K,iprint);
if iprint >= 2:
    print('zstore')
    print(zstore)

#### Compare Solutions from all three methods

In [ ]:
solution_compare(xstore,vstore[(K)*n:n*K],vstore,wstore,zstore,iprint)

### Add discretization error and compute error estimates

#### $K$  iterations computing subdomain contributions independently

#### Calculate the approximate solution by adding random error to the solution from each subdomain solve

In [ ]:
# print('\nPerforming C iteration with error \n')
if swrite == 1:
    svpert= randn(nK,1)
    writematrix(svpert,'svpert_add.csv','WriteMode','overwrite')

[vglobal_store,vaglobal_store,rglobal_store]= Citeration_additive_approx(C,f,alpha,n,p,K,sread,swrite,iprint)

if iprint >= 4:
    print('vglobal_store')
    disp(vglobal_store)
    print('vaglobal_store')
    disp(vaglobal_store)
    print('rglobal_store')
    disp(rglobal_store)    

In [ ]:
# Set adjoint RHS
psi= np.ones((n,1))

# Perform D iteration with error: add random error to each subdomain solve
v_store,va_store,r_store= Diteration_approx(D,f,n,p,K,iprint,sread,swrite,iparam)
if iprint >= 6:
    print(v_store)
    print(va_store)
    print(rstore)

#### Calculate the discretization error

In [ ]:
# Perform adjoint solve 
phi_store= Diteration_adjoint(D,psi,n,p,K,iprint)

# Compute error estimate as (r,phi)
qoi_discretization_error_estimate= r_store.T @ phi_store

# Compute error and error in QoI directly
vesoln=  v_store[n*p*K+n*(p-1):n*p*(K+1),0:1]
vsoln=  va_store[n*p*K+n*(p-1):n*p*(K+1),0:1]
verror= vesoln-vsoln

qoi_discretization_error= psi.T @ verror

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_disc = qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

#### Calculate the total error

In [ ]:
total_error= xstore-vsoln
qoi_total_error= psi.T@total_error

phi= np.linalg.solve(M.T,psi)
r= y-M@vsoln
qoi_total_error_estimate= r.T@phi

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_total= qoi_total_error_estimate/qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

#### Calculate the iteration error

In [ ]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

#### $K$ iterations as single matrix solve

In [ ]:
# Perform U iteration with error: add random error to linear system solution
zexact,zapprox,zresid= Uhsolve_multiplicative_approx(U,h,n,p,K,iprint,sread)

# Adjoint data (Adjoint system is of size npK)
psi_npK= np.zeros((n*p*K,1))
psi_npK[n*p*K-n:n*p*K,0]= psi.reshape(n)

#  Solve adjoint equation
phi_npK= np.linalg.solve(U.T,psi_npK)
if iprint >= 2:
    print(f'Norm of difference between phi_npK and phi_store = {np.linalg.norm(phi_npK-phi_store):13.6e}')

# Compute error estimate
qoi_discretization_error_estimate= phi_npK.T @ zresid

# Compute error and error in QoI directly
xsoln= zexact[n*p*K-n:n*p*K,0:1]
zsoln= zapprox[n*p*K-n:n*p*K,0:1]
zerror= xsoln-zsoln
qoi_discretization_error= psi.T @zerror

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_disc= qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

#### Calculate the total error

In [ ]:
total_error= xstore-zsoln
qoi_total_error= psi.T @ total_error

r= y-M @ zsoln
qoi_total_error_estimate= r.T@ phi

#### Calculate the effectivity ratio

In [ ]:
effectivity_ratio_total= qoi_total_error_estimate/qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

#### Calculate iteration error

In [ ]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')